In [3]:
from PIL import Image  # uses pillow
 


In [4]:
image_file = "imgs/train/c0/img_34.jpg"
im = Image.open(image_file)
print im.size   # return value is a tuple, ex.: (640, 480)

(640, 480)


In [5]:
image_file = "imgs/train/c3/img_5.jpg"
im = Image.open(image_file)
print im.size   # return value is a tuple, ex.: (1200, 800)

(640, 480)


In [6]:
import numpy as np
#from tqdm import tqdm
import tqdm
import cv2


In [7]:
img = cv2.imread(image_file, 0)
print img

[[44 44 44 ..., 27 30 83]
 [44 44 44 ..., 24 27 79]
 [44 44 44 ..., 23 28 76]
 ..., 
 [ 5  6  6 ..., 17 17 16]
 [ 5  6  6 ..., 18 17 16]
 [ 5  6  6 ..., 18 17 15]]


In [8]:
'''cv2.namedWindow("Image")
cv2.imshow("Image", img) 
cv2.waitKey (0)  
cv2.destroyAllWindows() '''

'cv2.namedWindow("Image")\ncv2.imshow("Image", img) \ncv2.waitKey (0)  \ncv2.destroyAllWindows() '

In [9]:
import numpy as np
from tqdm import tqdm
import cv2

from keras.models import *
from keras.applications import *
from keras.layers import *

Using TensorFlow backend.


In [10]:
height = 299
n = 10  # the size of the whole training set
y = np.zeros((n, 10), dtype=np.uint8) #
X = np.zeros((n, height, height, 3), dtype=np.uint8)

In [11]:
# For all kind of files, subdirectories included:
import os
list = os.listdir('imgs/train/c0') # dir is your directory path
number_files = len(list)
print number_files

2489


# Get the size of the whole training set

In [12]:
#Only files (avoiding subdirectories):
import os

onlyfiles = next(os.walk('imgs/train/c0/'))[2] #dir is your directory path as string
print len(onlyfiles)

2489


In [13]:
n = 0
for i in range(10):
    files = next(os.walk('imgs/train/c%d/'%i))[2]
    n += len(files)

print n   
# this is right!

22424


In [14]:
#X = cv2.imread()

In [15]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm
from sklearn.metrics import log_loss, accuracy_score
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
compare_loss={}
compare_accuracy={}

In [16]:
# function to convert image to array
def read_img(address, size):
    '''Read and resize image.
    Return image as numpy array, by normalizing the values'''
    img = image.load_img(address, target_size = size)
    img = image.img_to_array(img)
    return img
    

In [17]:
# function to convert labels to one hot encoding vector
def OneHotEncoded(y_train):
    y_t=np.zeros((len(y_train),Num_Class), dtype=int)
    for i,x in enumerate(y_train):
        #y_t[i][int(x)-1]=1
        y_t[i][int(x)] = 1
    return y_t

In [18]:
ls

Inception.ipynb            test.csv
Original.ipynb             train_i_bf.npy
README.md                  train_i_bf_.npy
Tutorial/                  train_vgg_bf.npy
VGG.ipynb                  train_x_bf.npy
Xception.ipynb             train_x_bf_.npy
driver_imgs_list.csv       valid_i_bf.npy
driver_imgs_list.csv.zip   valid_i_bf_.npy
imgs/                      valid_vgg_bf.npy
imgs.zip                   valid_x_bf.npy
labels.csv                 valid_x_bf_.npy
merge.ipynb                x_test.npy
reports/                   x_train.npy
sample_submission.csv      y_train.npy
sample_submission.csv.zip


In [19]:
import os
import csv
from shutil import copy2

In [20]:
INPUT_SIZE = 224
directory_lists=os.listdir("./imgs/train")

In [21]:
print(directory_lists)

['.DS_Store', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']


In [22]:
print(len(directory_lists))

11


In [23]:
labels = {}
data = {}

with open('./labels.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['id','class'])
    for i in range(0, len(directory_lists)):                        #'''directory_lists == files in /train == ['c0', 'c1', ... , 'c9']'''
        t = directory_lists[i][1:]
        imag=os.listdir("./imgs/train/"+str(directory_lists[i]))     #'''imag == files in /train/c# '''
        current_loc = "./imgs/train/"+str(directory_lists[i])+"/"    #'''./imgs/train/c#/'''
        for k in range(0, len(imag)):
            x=[current_loc+str(imag[k])]+[t]
            writer.writerow(x)

OSError: [Errno 20] Not a directory: './imgs/train/.DS_Store'

In [ ]:
##change here

In [ ]:
import os
import csv
from shutil import copy2
directory_lists=os.listdir("./imgs/test")
with open('./test.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['id'])
    for i in range(0, len(directory_lists)):                        #'''directory_lists == files in /train == ['c0', 'c1', ... , 'c9']'''
        x=["./imgs/test/"+directory_lists[i]]
        writer.writerow(x)

In [ ]:
ls

In [ ]:
data_dir = './'
labels = pd.read_csv(join(data_dir, 'labels.csv'))

In [26]:
INPUT_SIZE = 224
POOLING = 'avg'
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

x_train = np.zeros((len(labels), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
y_train= np.zeros((len(labels),), dtype='float32')

for i, img_id in tqdm(enumerate(labels['id'])):
    img = read_img(img_id, (INPUT_SIZE, INPUT_SIZE))
    x = preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_train[i] = x
    y_train[i]=int(labels['class'][i])
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))


22424it [02:43, 137.11it/s]


Train Images shape: (22424, 224, 224, 3) size: 3,375,439,872


change here


In [ ]:
test = pd.read_csv(join('./', 'test.csv'))

INPUT_SIZE = 224
POOLING = 'avg'
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

x_test = np.zeros((len(test), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')

for i, img_id in tqdm(enumerate(test['id'])):
    img = read_img(img_id, (INPUT_SIZE, INPUT_SIZE))
    x = preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_test[i] = x
print('Test Images shape: {} size: {:,}'.format(x_test.shape, x_test.size))



975it [00:05, 163.67it/s]

In [ ]:
np.save(open('x_train.npy', 'w'), x_train)
np.save(open('y_train.npy', 'w'), y_train)
np.save(open('x_test.npy', 'w'), x_test)

'''
#print(x_train)
for i in range(0, 22424, 1000):
    print(y_train[i])
print(len(y_train))

# y_train is int, why here is float
for i in range(0, len(x_test), 1000):
    print(x_test[i])
print(len(x_test))'''

In [ ]:
Num_Class=10  # number of classes in dataset
# convert labels to one hot encoding
y_train=OneHotEncoded(y_train)
# split data in to training and validation set
X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.33, random_state=42)


In [ ]:
'''base_model = ResNet50(input_tensor=Input((224, 224, 3)), weights='imagenet', include_top=False)

for layers in base_model.layers:
    layers.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.25)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(base_model.input, x)'''

In [ ]:
'''model.compile(optimizer='adadelta',
              loss='categorical_crossentropy',
              metrics=['accuracy'])'''

In [ ]:
'''model.fit(X_train, y_train, batch_size=16, epochs=5, validation_data=(X_val, y_val))'''

In [ ]:
vgg_bottleneck = VGG16(weights='imagenet', include_top=False, pooling=POOLING)
train_vgg_bf = vgg_bottleneck.predict(X_train, batch_size=32, verbose=1)
valid_vgg_bf = vgg_bottleneck.predict(X_val, batch_size=32, verbose=1)
test_vgg_bf = vgg_bottleneck.predict(X_val, batch_size=32, verbose=1)
print('VGG train bottleneck features shape: {} size: {:,}'.format(train_vgg_bf.shape, train_vgg_bf.size))
print('VGG valid bottleneck features shape: {} size: {:,}'.format(valid_vgg_bf.shape, valid_vgg_bf.size))

np.save(open('train_vgg_bf.npy', 'w'), train_vgg_bf)
np.save(open('valid_vgg_bf.npy', 'w'), valid_vgg_bf)
np.save(open('test_vgg_bf.npy', 'w'), valid_vgg_bf)

In [ ]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=147)
logreg.fit(train_vgg_bf, (y_train * range(Num_Class)).sum(axis=1))
valid_probs = logreg.predict_proba(valid_vgg_bf)
valid_preds = logreg.predict(valid_vgg_bf)
compare_loss['Vgg16']=log_loss(y_val, valid_probs)
compare_accuracy['Vgg16']=accuracy_score((y_val * range(Num_Class)).sum(axis=1), valid_preds)
print('Validation VGG LogLoss {}'.format(compare_loss['Vgg16']))
print('Validation VGG Accuracy {}'.format(compare_accuracy['Vgg16']))




In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(vgg_bottleneck).create(prog='dot', format='svg'))

In [ ]:

zip([x.name for x in vgg_bottleneck.layers], range(len(vgg_bottleneck.layers)))

In [ ]:
import matplotlib.pyplot as plt